In [12]:
import onnx
import onnx_graphsurgeon as gs
import numpy as np
# from rich import print

# Load the ONNX model
input_model_path = "complete.onnx"     # Replace with your model path
output_model_path = "simple_bev_model_fixed.onnx"

# Load the model with ONNX GraphSurgeon
model=onnx.load(input_model_path)
graph = gs.import_onnx(model)


onnx.checker.check_model(model)


def print_node(node):
    print(f"Node: {node.name}")
    print(f"Op: {node.op}")
    print(f"Inputs: {node.inputs}")
    print(f"Outputs: {node.outputs}")
    print(f"Attributes: {node.attrs}")


count=0
for node in graph.nodes:
    if node.op == "InstanceNormalization":
        print(f"Replacing ATen Instance Norm Node: {node.name}")
        count+=1
        print(f"Inputs {node.inputs[0]}")
        print(f"Scale {node.inputs[0]}")
        print(f"Bias {node.inputs[0]}")
        
        
print(f"Total ATen Instance Norm nodes found: {count}")
    

Replacing ATen Instance Norm Node: ATen_412_InstanceNorm
Inputs Variable (input.751): (shape=[6, 512, 28, 50], dtype=float32)
Scale Variable (input.751): (shape=[6, 512, 28, 50], dtype=float32)
Bias Variable (input.751): (shape=[6, 512, 28, 50], dtype=float32)
Replacing ATen Instance Norm Node: ATen_419_InstanceNorm
Inputs Variable (input.759): (shape=[6, 512, 28, 50], dtype=float32)
Scale Variable (input.759): (shape=[6, 512, 28, 50], dtype=float32)
Bias Variable (input.759): (shape=[6, 512, 28, 50], dtype=float32)
Replacing ATen Instance Norm Node: ATen_878_InstanceNorm
Inputs Variable (input.771): (shape=[1, 128, 200, 200], dtype=float32)
Scale Variable (input.771): (shape=[1, 128, 200, 200], dtype=float32)
Bias Variable (input.771): (shape=[1, 128, 200, 200], dtype=float32)
Replacing ATen Instance Norm Node: ATen_930_InstanceNorm
Inputs Variable (input.895): (shape=['unk__331', 128, 'unk__335', 'unk__336'], dtype=float32)
Scale Variable (input.895): (shape=['unk__331', 128, 'unk__3

In [13]:
for node in graph.nodes:
    if node.op == "InstanceNormalization":
        print(f"Replacing ATen Instance Norm Node: {node.name}")
        # print(node.inputs[0])
        
        num_channels = node.inputs[0].shape[1]

        scale = gs.Constant(name=f"{node.name}_scale", values=np.ones(num_channels, dtype=np.float32))
        bias = gs.Constant(name=f"{node.name}_bias", values=np.zeros(num_channels, dtype=np.float32))
        
        node.inputs[1]=scale
        node.inputs[2]=bias
        

Replacing ATen Instance Norm Node: ATen_412_InstanceNorm
Replacing ATen Instance Norm Node: ATen_419_InstanceNorm
Replacing ATen Instance Norm Node: ATen_878_InstanceNorm
Replacing ATen Instance Norm Node: ATen_930_InstanceNorm
Replacing ATen Instance Norm Node: ATen_938_InstanceNorm
Replacing ATen Instance Norm Node: ATen_946_InstanceNorm
Replacing ATen Instance Norm Node: ATen_953_InstanceNorm
Replacing ATen Instance Norm Node: ATen_978_InstanceNorm
Replacing ATen Instance Norm Node: ATen_961_InstanceNorm
Replacing ATen Instance Norm Node: ATen_969_InstanceNorm


In [14]:
count=0
for node in graph.nodes:
    if node.op == "InstanceNormalization":
        print(f"Replacing ATen Instance Norm Node: {node.name}")
        count+=1
        print(f"Inputs {node.inputs[0]}")
        print(f"Scale {node.inputs[0]}")
        print(f"Bias {node.inputs[0]}")
        
        
print(f"Total ATen Instance Norm nodes found: {count}")

model=gs.export_onnx(graph)
onnx.save(model, "size_fixed.onnx")

Replacing ATen Instance Norm Node: ATen_412_InstanceNorm
Inputs Variable (input.751): (shape=[6, 512, 28, 50], dtype=float32)
Scale Variable (input.751): (shape=[6, 512, 28, 50], dtype=float32)
Bias Variable (input.751): (shape=[6, 512, 28, 50], dtype=float32)
Replacing ATen Instance Norm Node: ATen_419_InstanceNorm
Inputs Variable (input.759): (shape=[6, 512, 28, 50], dtype=float32)
Scale Variable (input.759): (shape=[6, 512, 28, 50], dtype=float32)
Bias Variable (input.759): (shape=[6, 512, 28, 50], dtype=float32)
Replacing ATen Instance Norm Node: ATen_878_InstanceNorm
Inputs Variable (input.771): (shape=[1, 128, 200, 200], dtype=float32)
Scale Variable (input.771): (shape=[1, 128, 200, 200], dtype=float32)
Bias Variable (input.771): (shape=[1, 128, 200, 200], dtype=float32)
Replacing ATen Instance Norm Node: ATen_930_InstanceNorm
Inputs Variable (input.895): (shape=['unk__331', 128, 'unk__335', 'unk__336'], dtype=float32)
Scale Variable (input.895): (shape=['unk__331', 128, 'unk__3

In [ ]:
# input_tensor = aten1.inputs[0]

# # Extract epsilon and momentum values
# epsilon = 1e-5

# scale = gs.Constant(name=f"{node.name}_scale", values=np.array(1, dtype=np.float32))
# bias = gs.Constant(name=f"{node.name}_bias", values=np.array(0, dtype=np.float32))

# inst_norm_output = gs.Variable(name=f"{node.name}_InstanceNorm_output", dtype=np.float32)

# # Create InstanceNormalization node
# inst_norm_node = gs.Node(
#     op="InstanceNormalization",
#     name=f"{node.name}_InstanceNorm",
#     inputs=[input_tensor, scale, bias],
#     outputs=[inst_norm_output],
#     attrs={"epsilon": epsilon}
# )

# # Add the new node to the graph
# graph.nodes.remove(aten1)
# graph.nodes.append(inst_norm_node)


# relu = [node for node in graph.nodes if node.name == "Relu_970"][0]

# relu.inputs = inst_norm_node.outputs

# graph.cleanup().toposort()


NameError: name 'aten1' is not defined

In [ ]:
# # # Clean up and save the modified model
# # graph.cleanup().toposort()

model=gs.export_onnx(graph)
onnx.checker.check_model(model)
model = onnx.shape_inference.infer_shapes(model)
inferred_model_path = "final.onnx"
onnx.save(model, inferred_model_path)
print(f"Shape-inferred model saved at {inferred_model_path}")


Shape-inferred model saved at final.onnx


## ONNX Runtime Inference

In [ ]:
import onnxruntime as ort

ort_session = ort.InferenceSession("size_fixed.onnx")

print([x.name for x in ort_session.get_inputs()])

print([x.name for x in ort_session.get_outputs()])


['rgb_camXs', 'pix_T_cams', 'cam0_T_camXs']
['output_0', 'feat_bev_e', 'seg_bev_e', 'center_bev_e', 'offset_bev_e']


In [ ]:
input_shapes = [x.shape for x in ort_session.get_inputs()]
input_shapes

input_tensors = [np.random.random(shape).astype(np.float32) for shape in input_shapes]

inputs =dict(zip([x.name for x in ort_session.get_inputs()], input_tensors))

In [ ]:
output_tensors = ort_session.run(None,inputs)

[(1, 128, 200, 200), (1, 128, 200, 200), (1, 1, 200, 200), (1, 1, 200, 200), (1, 2, 200, 200)]
